In [ ]:
!rm -r sample_data/

rm: cannot remove 'sample_data/': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/colab_data/diplom/'
path_data = path + 'data/'

In [ ]:
# Раскомментировать, если данные упакованы в zip архив

# !unzip $path'train.zip' -d $path
# !unzip $path'test_kaggle.zip' -d $path
# !unzip $path'skillbox-computer-vision-project.zip' -d $path''

In [69]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os

from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [37]:
# Добавляем классы в словарь
classes = {0: 'anger',\
           1: 'contempt',\
           2: 'disgust',\
           3: 'fear',\
           4: 'happy',\
           5: 'neutral',\
           6: 'sad',\
           7: 'surprise'}

In [ ]:
#Константы

BATCH_SIZE = 128
IMG_SIZE = 48
EPOCHS = 100
STEPS_PER_EPOCH = 30

In [ ]:
#Модель

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dense(8, activation='softmax'))

#На случай, если необходимо заморозить слои

# for l in model.layers[:-1]:
  # l.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         1

In [ ]:
# Создаём поток из фотографий
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        path_data+'train',
        target_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE,
        color_mode="grayscale",
        class_mode='categorical')

Found 14237 images belonging to 8 classes.


In [ ]:
# Обучаем и сохраняем веса модели
mcp = ModelCheckpoint(path + 'mdl_wts.hdf5',\
                      save_best_only=True,\
                      monitor='accuracy')
opt = Adam(lr=0.0001, decay=1e-6)


model.compile(loss='categorical_crossentropy',\
              optimizer=opt,\
              metrics=['accuracy'])
model.fit(train_generator,\
          epochs=EPOCHS,\
          steps_per_epoch=STEPS_PER_EPOCH,\
          callbacks=[mcp])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
30/30 [==============================] - 14s 442ms/step - loss: 1.3554 - accuracy: 0.5068
Epoch 2/100
30/30 [==============================] - 13s 434ms/step - loss: 0.8353 - accuracy: 0.6802
Epoch 3/100
30/30 [==============================] - 13s 438ms/step - loss: 0.7258 - accuracy: 0.7068
Epoch 4/100
30/30 [==============================] - 13s 447ms/step - loss: 0.6623 - accuracy: 0.7474
Epoch 5/100
30/30 [==============================] - 13s 420ms/step - loss: 0.6285 - accuracy: 0.7552
Epoch 6/100
30/30 [==============================] - 13s 449ms/step - loss: 0.6400 - accuracy: 0.7552
Epoch 7/100
30/30 [==============================] - 13s 437ms/step - loss: 0.5776 - accuracy: 0.7755
Epoch 8/100
30/30 [==============================] - 13s 448ms/step - loss: 0.5949 - accuracy: 0.7734
Epoch 9/100
30/30 [==============================] - 14s 449ms/step - loss: 0.5183 - accuracy: 0.7899
Epoch 10/100
30/30 [==============================] - 14s 447ms/step - loss: 0.537

In [70]:
# Оптимизируем модель с помощью TensorRT
model.load_weights(path + 'mdl_wts.hdf5')

model.save(path+'saved_model')

converter = trt.TrtGraphConverterV2(input_saved_model_dir=path+'saved_model')
converter.convert()
converter.save(path+'saved_model_trt')

model_trt = tf.keras.models.load_model(path+'saved_model_trt')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/colab_data/diplom/saved_model/assets
INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: /content/drive/MyDrive/colab_data/diplom/saved_model_trt/assets


In [71]:
TEST_BATCH_SIZE = 100

out = []

imgs = []
files = [] 

# Прогоняем все файлы для теста

for filepath in os.listdir(path_data+'test_kaggle/'):
  files.append(filepath)
  imgs.append((np.array(load_img(path_data+'test_kaggle/{0}'.format(filepath),\
                       color_mode='grayscale',\
                       target_size=(IMG_SIZE,IMG_SIZE)), dtype=np.float32) / 255.)[..., None])

  if len(imgs) == TEST_BATCH_SIZE:
    imgs = np.array(imgs, dtype=np.float32)

    predictions = model_trt(imgs)
    predictions = [classes[predict] for predict in np.argmax(predictions, axis=1)]

    out += list(zip(files, predictions))

    imgs = []
    files = []

In [72]:
# Записываем результат в csv файл

out = pd.DataFrame(out, columns=['image_path', 'emotion'])
out['sort_val'] = out.image_path.apply(lambda x: int(x.replace('.jpg', '')))
out = out.sort_values('sort_val')
out = out.drop('sort_val', 1)

compression_opts = dict(method='zip', archive_name='out.csv') 

out.to_csv(path+'out.zip', index=False, compression=compression_opts)  